In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import warnings

warnings.simplefilter("ignore", category=Warning)

In [42]:
# Set the global seed
tf.random.set_seed(42)

In [43]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Time Series/Amazon_Stock_Price.csv', index_col=0)


In [44]:
df.head()

,Date,Open,High,Low,Close,Volume
0,2000-01-03,4.075000,4.478125,3.952344,4.468750,322352000
1,2000-01-04,4.268750,4.575000,4.087500,4.096875,349748000
2,2000-01-05,3.525000,3.756250,3.400000,3.487500,769148000
3,2000-01-06,3.565625,3.634375,3.200000,3.278125,375040000
4,2000-01-07,3.350000,3.525000,3.309375,3.478125,210108000


In [45]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Time Series/Amazon_Stock_Price.csv', usecols=lambda column: column != 'Unnamed: 0')

In [46]:
df.head()

,Date,Open,High,Low,Close,Volume
0,2000-01-03,4.075000,4.478125,3.952344,4.468750,322352000
1,2000-01-04,4.268750,4.575000,4.087500,4.096875,349748000
2,2000-01-05,3.525000,3.756250,3.400000,3.487500,769148000
3,2000-01-06,3.565625,3.634375,3.200000,3.278125,375040000
4,2000-01-07,3.350000,3.525000,3.309375,3.478125,210108000


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6050 entries, 0 to 6049
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    6050 non-null   object 
 1   Open    6050 non-null   float64
 2   High    6050 non-null   float64
 3   Low     6050 non-null   float64
 4   Close   6050 non-null   float64
 5   Volume  6050 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 283.7+ KB


In [48]:
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by='Date', inplace=True)
df.set_index('Date', inplace=True)

In [49]:
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2000-01-03,4.075000,4.478125,3.952344,4.468750,322352000
2000-01-04,4.268750,4.575000,4.087500,4.096875,349748000
2000-01-05,3.525000,3.756250,3.400000,3.487500,769148000
2000-01-06,3.565625,3.634375,3.200000,3.278125,375040000
2000-01-07,3.350000,3.525000,3.309375,3.478125,210108000


In [50]:
# Feature engineering
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day

In [51]:
df['season'] = df['month'].apply(lambda month: 1 if month in [12, 1, 2] else 2
                                 if month in [3, 4, 5] else 3 if month in [6, 7, 8] else 4)

In [52]:
# Correlation analysis to select top features
corr_matrix = df.corr().abs()
target = 'Close'
normalized_corr = (corr_matrix[target] - corr_matrix[target].min()) / (corr_matrix[target].max() - corr_matrix[target].min())


In [53]:
df.head()

,Open,High,Low,Close,Volume,year,month,day,season
Date,,,,,,,,,
2000-01-03,4.075000,4.478125,3.952344,4.468750,322352000,2000,1,3,1
2000-01-04,4.268750,4.575000,4.087500,4.096875,349748000,2000,1,4,1
2000-01-05,3.525000,3.756250,3.400000,3.487500,769148000,2000,1,5,1
2000-01-06,3.565625,3.634375,3.200000,3.278125,375040000,2000,1,6,1
2000-01-07,3.350000,3.525000,3.309375,3.478125,210108000,2000,1,7,1


In [54]:
normalized_corr

Open      0.999783
High      0.999898
Low       0.999903
Close     1.000000
Volume    0.301610
year      0.828067
month     0.036163
day       0.000000
season    0.027937
Name: Close, dtype: float64

In [55]:
# Select top 5 features correlated with the target
n = 5
top_features = normalized_corr.sort_values(ascending=False).index[1:n+1]
top_features = list(top_features)


In [56]:
top_features

['Low', 'High', 'Open', 'year', 'Volume']

In [57]:
top_features.append('Close')

In [58]:
top_features

['Low', 'High', 'Open', 'year', 'Volume', 'Close']

In [59]:
# Function to create sequences for the RNN model
def create_sequences_optimized(data, seq_length, target_idx):
    data_values = data.values.astype('float32')
    num_samples = len(data) - seq_length
    num_features = data.shape[1]

    xs = np.empty((num_samples, seq_length, num_features), dtype='float32')
    ys = np.empty(num_samples, dtype='float32')

    for i in range(num_samples):
        xs[i] = data_values[i:i+seq_length]
        ys[i] = data_values[i+seq_length, target_idx]

    return xs, ys

In [60]:
# Prepare the data
data = df[top_features].copy()
scalers = {}
for feature in top_features:
    scaler = MinMaxScaler(feature_range=(0, 1))
    data[feature] = scaler.fit_transform(data[[feature]])
    scalers[feature] = scaler

In [61]:
# Parameters for the LSTM model
# Uncomment and pass in values for sequence and batch_size
sequence = 12
batch_size = 32
target_column = 'Close'


In [62]:
# Uncomment the code below to create sequences for the RNN model
target_idx = top_features.index(target_column)
seq_length = sequence
X, y = create_sequences_optimized(data, seq_length, target_idx)

In [63]:
from sklearn.model_selection import train_test_split

In [64]:
# Uncomment the code below to split the data using train_test_split (pass in the value for test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)







## **Instructions for Fellows:**
##### As part of this exercise, you are required to create the RNN model.
##### Follow the provided guidelines and use the specified parameters above.
##### Implement the RNN model, train it on the given data and save the trained model.

In [65]:
from tensorflow.keras import backend as K

In [66]:
def r2_score(y_true,y_pred):
  ss_res = K.sum(K.square(y_true - y_pred))
  ss_tot = K.sum(K.square(y_true - K.mean(y_true)))
  return 1-ss_res/(ss_tot+K.epsilon())

In [72]:
#RNN model
# Build the RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(seq_length, 6), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(1, activation='relu'))

In [81]:
# Compile the model
model.compile(optimizer='adam', loss='mse')

In [82]:
# Set up ModelCheckpoint callback to save the entire model
checkpoint = ModelCheckpoint('model.h5', save_best_only=True, save_weights_only=False, verbose=1)




In [83]:
# Train the model


history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[checkpoint])

Epoch 1/10
151/151 [==============================] - ETA: 0s - loss: 3.8453e-05
Epoch 1: val_loss improved from inf to 0.00060, saving model to model.h5
151/151 [==============================] - 17s 86ms/step - loss: 3.8453e-05 - val_loss: 6.0361e-04
Epoch 2/10
151/151 [==============================] - ETA: 0s - loss: 2.9863e-05
Epoch 2: val_loss did not improve from 0.00060
151/151 [==============================] - 9s 58ms/step - loss: 2.9863e-05 - val_loss: 7.7328e-04
Epoch 3/10
151/151 [==============================] - ETA: 0s - loss: 3.2167e-05
Epoch 3: val_loss did not improve from 0.00060
151/151 [==============================] - 6s 42ms/step - loss: 3.2167e-05 - val_loss: 8.8480e-04
Epoch 4/10
151/151 [==============================] - ETA: 0s - loss: 3.5049e-05
Epoch 4: val_loss did not improve from 0.00060
151/151 [==============================] - 9s 61ms/step - loss: 3.5049e-05 - val_loss: 0.0017
Epoch 5/10
150/151 [============================>.] - ETA: 0s - loss: 2.2

In [86]:
model.save('model.h5')

In [88]:
# Uncomment the code below to load the best model and make predictions
model = load_model('model.h5')
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

151/151 [==============================] - 5s 30ms/step


In [89]:
# Uncomment the code below to inverse transform the predictions and actual values
y_test_actual = scalers[target_column].inverse_transform(y_test.reshape(-1, 1)).flatten()
y_pred_actual = scalers[target_column].inverse_transform(y_pred).flatten()
y_train_actual = scalers[target_column].inverse_transform(y_train.reshape(-1, 1)).flatten()
y_train_pred_actual = scalers[target_column].inverse_transform(y_train_pred).flatten()



In [91]:
from sklearn.metrics import r2_score

In [92]:
# Uncomment the code below to calculate R2 scores for train and test sets
r2_train = r2_score(y_train_actual, y_train_pred_actual)
r2_test = r2_score(y_test_actual, y_pred_actual)

In [93]:
# Uncomment the code below to print the results
print(f'Inverse Test Min: {y_test_actual.min()}, Inverse Test Max: {y_test_actual.max()}')
print(f'Inverse Pred Min: {y_pred_actual.min()}, Inverse Pred Max: {y_pred_actual.max()}')
print(f'R² Score For Train Data: {round(r2_train * 100, 2)}%')
print(f'R² Score For Test Data: {round(r2_test * 100, 2)}%')

Inverse Test Min: 81.81999969482422, Inverse Test Max: 186.57049560546875
Inverse Pred Min: 85.6744155883789, Inverse Pred Max: 183.95579528808594
R² Score For Train Data: 99.8%
R² Score For Test Data: 98.72%
